In [94]:
import pandas_datareader as web
import pandas as pd
import time
from datetime import datetime
import datetime as dt
import numpy as np
import math
import statistics
import matplotlib.pyplot as plt
from matplotlib import style

In [95]:
#導入指數資料
xls = pd.ExcelFile (r'C:\Users\nfk_2\Desktop\論文\thesis.xlsx')
LEGATRUU = pd.read_excel(xls, 'LEGATRUU')
LUATTRUU = pd.read_excel(xls, 'LUATTRUU')
LF98TRUU = pd.read_excel(xls, 'LF98TRUU')
LUMSTRUU = pd.read_excel(xls, 'LUMSTRUU')
LUACTRUU = pd.read_excel(xls, 'LUACTRUU')
LBUSTRUU = pd.read_excel(xls, 'LBUSTRUU')
USGB090Y = pd.read_excel(xls, 'USGB090Y')
USGG5YR = pd.read_excel(xls, 'USGG5YR')
H15T20Y = pd.read_excel(xls, 'H15T20Y')
MXWD = pd.read_excel(xls, 'MXWD')
SENSEX = pd.read_excel(xls, 'SENSEX')
KOSPI = pd.read_excel(xls, 'KOSPI')
TWSE = pd.read_excel(xls, 'TWSE')
M2WO = pd.read_excel(xls, 'M2WO')
SPX = pd.read_excel(xls, 'SPX')
SPTSX = pd.read_excel(xls, 'SPTSX')
TPX = pd.read_excel(xls, 'TPX')
ASX = pd.read_excel(xls, 'ASX')

In [96]:
#導入生命表
xlss = pd.ExcelFile (r'C:\Users\nfk_2\Desktop\論文\life table\US.xlsx')
male = pd.read_excel(xlss, 'Male')
female = pd.read_excel(xlss, 'Female')

In [97]:
#蒙地卡羅模擬
def expected_r(ID,yy,mm,dd):
    start = dt.datetime(yy-20,mm,dd)     #here: 用過去20年模擬
    end = dt.datetime(yy,mm,dd)    
    
    mask = (ID['Date'] >= start) & (ID['Date'] <= end)
    prices = ID.loc[mask]['Price']
    #print(prices)
    
    returns = prices.pct_change()
    #print(returns)

    last_price = prices[-1:].item()

    num_simulation = 10000             #共模擬一萬次
    num_days = 252

    simulation_df = pd.DataFrame()
    
    for x in range(num_simulation):
        count = 0
        sigma = returns.std()
        mu = returns.mean()
    
        price_series = []
    
        price = last_price * np.exp((mu - sigma**2/2) + sigma * np.random.normal(0,1))
        price_series.append(price)
    
        for y in range(num_days):
            if count == 251:
                break
            price = price_series[count] * np.exp((mu - sigma**2/2) + sigma * np.random.normal(0,1))
            price_series.append(price)
            count += 1
    
        simulation_df[x] = price_series

#     fig = plt.figure()
#     plt.plot(simulation_df)
#     plt.axhline(y = last_price, color = 'r', linestyle = '-')
#     plt.xlabel('Day')
#     plt.ylabel('Price')
#     plt.show()    
    
    #print(simulation_df)

    rtn = ((simulation_df.iloc[-1].mean() - last_price)/last_price)
    return(rtn)

In [99]:
#投資組合報酬模擬
def port_r(S,YY,MM,DD):    #,B
    stock = expected_r(S,YY,MM,DD)
    #bond = expected_r(B,YY,MM,DD)
    #final = (0.5 * stock + 0.5 * bond)
    return(stock)

In [100]:
###上下限報酬
def limit_r(ID,yy,mm,dd):
    start = dt.datetime(yy-20,mm,dd)          #here
    end = dt.datetime(yy,mm,dd)    
    
    mask = (ID['Date'] >= start) & (ID['Date'] <= end)
    prices = ID.loc[mask]['Price']    
    returns = prices.pct_change()
    
    r = (1 + returns.mean())**252 - 1
    sd = returns.std() * math.sqrt(252)
    
    return(r,sd)

In [102]:
#實際每年報酬
def real(ID, yy, mm, dd):      #asset1  only
    start = dt.datetime(yy,mm,dd)     
    end = dt.datetime(yy + 1,mm,dd)    
    
    mask = (ID['Date'] >= start) & (ID['Date'] <= end)
    prices = ID.loc[mask]['Price']
    rn = ((prices[-1:].item()/prices[:1].item())-1)
        
    return(rn)

In [103]:
#結合模擬和實際資料
def combine(lumpsum,retire,asset1,year,month,day):     ###,asset2
    eachpay = 0
    flow = []
    yeartab = []
    agetab = []
    livingtab = []
    rcaltab = []
    rup = []
    rlo = []
    usedtab = []
    lefttab = []
    endtab = []
    fixtab = []
    fixleft = []
    fixend = []
    invest = lumpsum
    
    
    realr = []

    for i in range(120-retire):             #according to the maximum life
        rcal = port_r(asset1,(year + i), month, day)    ###,asset2
        living = male.loc[male['Age'] == retire+i]['Life'].item()
        upper = limit_r(asset1,(year + i), month, day)[0] + 2 * limit_r(asset1,(year + i), month, day)[1]/math.sqrt(252)
        lower = limit_r(asset1,(year + i), month, day)[0] - 2 * limit_r(asset1,(year + i), month, day)[1]/math.sqrt(252)
        used_r = min(max(lower,rcal),upper)
        eachpay = round(-np.pmt(used_r, living, invest, 0, when = 1), 2)
        
        yeartab.append(year+i)
        realr.append(round(real(asset1,year + i, month, day)*100,2))   ##asset1 only
        rup.append(round(upper*100,2))
        rlo.append(round(lower*100,2))
        usedtab.append(round(used_r*100,2))
        flow.append(eachpay)
        agetab.append(retire+i)
        livingtab.append(living)
        rcaltab.append(round(rcal*100,2))
        lefttab.append(round((invest - eachpay),2))
        endtab.append(round((invest - eachpay) * (1 + real(asset1,year + i, month, day)),2))
        invest = (invest - eachpay) * (1 + real(asset1,year + i, month, day)) 

    return(yeartab,agetab,livingtab,rcaltab,rup,rlo,usedtab,flow,realr,lefttab,endtab)

In [104]:
#導出資料表
output_table = pd.DataFrame(combine(1000000,65,SPX,1965,10,22)).transpose()   ###,LBUSTRUU
output_table.columns = ['Year','Age', 'E.Life', 'Sim.R','Upper.R','Lower.R','Used.R',
                        'Annuity', 'Real.R','Left(beg)','Left(end)']
output_table['Year'] = output_table['Year'].astype(int)
output_table['Age'] = output_table['Age'].astype(int)

output_table

C:\Users\nfk_2\anaconda3\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: numpy.pmt is deprecated and will be removed from NumPy 1.20. Use numpy_financial.pmt instead (https://pypi.org/project/numpy-financial/).


,Year,Age,E.Life,Sim.R,Upper.R,Lower.R,Used.R,Annuity,Real.R,Left(beg),Left(end)
0,1965,65,17.89,9.73,11.29,8.22,9.73,109453.97,-14.99,890546.03,757031.90
1,1966,66,17.18,9.45,10.88,7.95,9.45,82946.40,21.63,674085.50,819870.89
2,1967,67,16.47,10.15,11.58,8.75,10.15,94865.32,10.12,725005.57,798376.50
3,1968,68,15.77,10.55,11.72,8.98,10.55,95922.14,-6.45,702454.36,657178.06
4,1969,69,15.07,10.32,11.58,8.89,10.32,79593.76,-14.77,577584.30,492272.09
5,1970,70,14.39,8.36,9.46,6.74,8.36,55439.69,14.55,436832.40,500382.14
6,1971,71,13.71,8.08,9.44,6.76,8.08,57091.41,14.38,443290.73,507015.80
7,1972,72,13.05,8.63,9.93,7.24,8.63,60986.65,-1.08,446029.15,441219.23
8,1973,73,12.40,8.51,9.84,7.12,8.51,54336.76,-33.01,386882.47,259185.74
9,1974,74,11.76,5.08,6.41,3.46,5.08,28373.08,24.04,230812.66,286298.60
